In [1]:
# vs.py  (voting system code)
# Ronald L. Rivest and Emily Shen
# Updated: 3/29/2010
# 
# Given a set of preference ballots, compute GT winner, etc...
# This is the code used for our paper on the GT voting system.  

"""
** Author:  Ronald L. Rivest and Emily Shen
** Address: Room 32G-692 Stata Center 
**          32 Vassar Street 
**          Cambridge, MA 02139
** Email:   rivest@mit.edu, eshen@csail.mit.edu
** Date:    3/27/2010
**
** (The following license is known as "The MIT License")
** 
** Copyright (c) 2010 Ronald L. Rivest and Emily Shen
** 
** Permission is hereby granted, free of charge, to any person obtaining a copy
** of this software and associated documentation files (the "Software"), to deal
** in the Software without restriction, including without limitation the rights
** to use, copy, modify, merge, publish, distribute, sublicense, and/or sell
** copies of the Software, and to permit persons to whom the Software is
** furnished to do so, subject to the following conditions:
** 
** The above copyright notice and this permission notice shall be included in
** all copies or substantial portions of the Software.
** 
** THE SOFTWARE IS PROVIDED "AS IS", WITHOUT WARRANTY OF ANY KIND, EXPRESS OR
** IMPLIED, INCLUDING BUT NOT LIMITED TO THE WARRANTIES OF MERCHANTABILITY,
** FITNESS FOR A PARTICULAR PURPOSE AND NONINFRINGEMENT. IN NO EVENT SHALL THE
** AUTHORS OR COPYRIGHT HOLDERS BE LIABLE FOR ANY CLAIM, DAMAGES OR OTHER
** LIABILITY, WHETHER IN AN ACTION OF CONTRACT, TORT OR OTHERWISE, ARISING FROM,
** OUT OF OR IN CONNECTION WITH THE SOFTWARE OR THE USE OR OTHER DEALINGS IN
** THE SOFTWARE.
**
** (end of license)
"""

import math
import os
import random
import string
import sys

import game_cvxopt                  # LP and QP solvers for two-person zero-sum games

########################################################################################
### Some global variables...
########################################################################################

indent = "    "                      # standard indent amount within each output section

########################################################################################
### ALTERNATIVES (CANDIDATES)
########################################################################################

"""
An alternative (or candidate) is represented by an integer or a string.
The only restrictions are that a candidate name should not start with
a left parenthesis, include an embedded blank, an embedded equals sign, or be null.

Examples of acceptable alternative names:
    0   1   A  B  Alice  MaryJones  Bob_Smith  #1  #2

We often use the variable A to denote the set of alternatives for an election.
"""

def print_alternatives(A,election_ID):
    """
    Print out list of candidates, nicely, in sorted order.
    """
    print("%s: Alternatives:"%election_ID)
    print(indent+string.join([str(alt) for alt in sorted(A)]))



In [2]:
########################################################################################
### BALLOTS (TUPLES OF CANDIDATES)
########################################################################################

"""
Since this code is about preferential voting, a ballot consists of a nonempty tuple
(Python immutable list) of candidates.  The first element indicates the most-preferred
candidate.  A ballot is a tuple rather than a list so that it can be used as a hash
table (dict) key.  A ballot in some cases may be "short" (i.e. truncated), and only give 
the initial portion of the voter's preferences.

A ballot may contain an equals sign ("=") as one element.  This is not a candidate,
but indicates that the following candidate is to be ranked the same as the previous
candidate.  Some procedures, such as IRV, ignore the "=" signs.  Note that ballots
("A", "B", "C", "=", "D") and ("A", "B") are semantically equivalent when truncated
ballots are allowed (and the candidates are "A", "B", "C", "D"), but we don't
bother to determine when a profile contains two such equivalent ballots.

Examples of ballots:
    (2, 0, 1)
    ("A", "C", "B")
    ("Mary_Smith", "Bob_Jones", "Sarah_Wilson")
    ("Harry",)
    ("A", "=", "B", "C", "=", "D")

"""

def perms(A,rlo=None,rhi=None):
    """
    This routine is for generating a list of "all possible" ballots 
    for a given set  A of alternatives.  (It does not produce ballots
    containing equals signs.)

    There are typically two cases, depending on whether "short" ballots
    are allowed.

    To get all possible permutations of A: perm(A)

    To get all permutations of lengths between rlo and rhi: perms(A,rlo,rhi)
    (Here perms(A,1,len(A)) gives all nonempty ballots on A, including short
    ones.)

    If A is an integer k, it is interpreted as A = range(k) = [0,1,...,k-1].

    Return all r-permutations of set A, for all r such that
        rlo <= r <= rhi
    If rlo is not given, then rlo = len(A)
       so perms(A) gives all full permutations of A.
    If rhi is not given, then rhi = rlo.

    Examples:

    A = [0,1,2,3]
    perms(A,1,2) = 
      [(0,), (0, 1), (0, 2), (0, 3), 
       (1,), (1, 0), (1, 2), (1, 3), 
       (2,), (2, 0), (2, 1), (2, 3), 
       (3,), (3, 0), (3, 1), (3, 2)]

    perms(3) = [(0, 1, 2), (0, 2, 1), (1, 0, 2), (1, 2, 0), (2, 0, 1), (2, 1, 0)]
    """

    if type(A)==type(0):  # int
        A = range(A)
    if rlo==None:
        rlo = len(A)
    if rlo==0:
        ans = [tuple()]
    else:
        ans = []
    if rhi==None: 
        rhi = rlo
    if rhi==0:
        return ans
    for i in range(len(A)):
        x = A[i]
        Awox = A[:i] + A[i+1:]              # A without x
        L1 = perms(Awox,max(0,rlo-1),rhi-1)
        L2 = [ (x,)+p for p in L1 ]
        ans.extend( L2 )
    return sorted(ans)

def ballot_OK(ballot,A=None):
    """
    Check the given ballot (a tuple):
       -- has all entries distinct  (except OK to have multiple equals signs)
       -- has no syntactically illegal entries:
            entries are non-null, have no whitespace or equal signs, 
            and don't start with "(".  (Exception: bare "=" is OK.)
       -- [if A is not null]: check that all items are in A
    return True if the ballot looks OK
    """
    if len(ballot)==0:                         # zero-length ballot is legal
        return True
    ballot = [ str(a) for a in ballot ]        # convert all to string form for checking
    for (x,y) in zip(sorted(ballot),sorted(ballot)[1:]):
        if x==y and x!="=": return False       # duplicate found (other than duplicate "=")
    if A != None:                              # if A given, all items must be in A
        for item in ballot:
            if item != "=" and item not in A:
                return False
    for (position,item) in enumerate(ballot):
        if len(item) == 0:                     # null string not allowed as item
            return False
        if item[0]=="(":                       # item can't start with a parenthesis
            return False
        for ws in string.whitespace:           # item can't contain whitespace
            if item.find(ws)>=0:
                return False
        if len(item)>1 and item.find('=')>=0:  # item can't have embedded equals signs
            return False
        if item=="=":
            if position==0:                    # ballot can't start with equals sign
                return False
            if position==len(ballot)-1:        # ballot can't end with an equals sign
                return False
            if ballot[position-1]=="=":        # ballot can't have two equals signs in a row
                return False
    return True                                # Ballot is legal

def ballots_for(B,c,elim):
    """
    Return list of all ballots in B voting for c, once candidates 
    in list elim are eliminated.  (The returned ballots still have
    the eliminated candidates; they are just ignored when determining
    if the ballot is for c.)

    B = list of ballots; each ballot is list of distinct candidates

    c = a specific candidate (e.g. 0)

    elim = list of previously eliminated candidates 

    This routine flushes any "=" appearing in a ballot; IRV treats
    ballots as if the "=" signs didn't appear in them at all.

    Example:
       ballots_for(perms(3), 0, [1]) = [(0, 1, 2), (0, 2, 1), (1, 0, 2)]
    """
    ans = []
    for ballot in B:
        filtered_ballot = filter( lambda x: x != "=" and x not in elim, ballot)
        if len(filtered_ballot)>0 and filtered_ballot[0] == c:
            ans.append(ballot)
    return ans


In [10]:
########################################################################################
### PROFILES AND INPUT ROUTINES
########################################################################################
"""
A profile is a set of ballots, with multiplicities.
A profile is represented as a dict P mapping ballots to nonnegative integer multiplicities.
"""

def alternatives_in_profile(P):
    """
    Return the list of alternatives appearing in profile P, in sorted order.
    """
    A = set()
    for ballot in P:
        for alternative in ballot:
            if alternative != "=":
                A.add(alternative)
    return sorted(list(A))

def number_of_ballots_in_profile(P):
    """
    Return number of ballots in profile P.
    """
    return sum([P[ballot] for ballot in P])

def first_choice_counts(A,P):
    """
    Return list giving first-choice counts, in decreasing order by count
    """
    count = { }
    for a in A:
        count[a] = 0
    for ballot in P:
        if len(ballot)>0:
            a = ballot[0]
            count[a] += P[ballot]
    L = [ (count[a],a) for a in A ]
    L = sorted(L)
    L.reverse()
    return L

def print_first_choice_counts(A,P,election_ID):
    """
    Print out a line containing the first choice counts for each candidate.
    """
    print("%s: Count of number of times each candidate was given as first choice."%election_ID)
    L = first_choice_counts(A,P)
    line = indent
    for (cnt,a) in L:
        line = line + "%s (%d) "%(a,cnt)
    print(line)

def parse_ballot_line(line):
    """
    line = line of text representing a single ballot, with multiplicities.
    default count is 1, but any nonnegative count can be given in parentheses.
    Recommendation is to give count *last*, but it could be anywhere.
    Return (ballot,count)

    Input examples:
       "Charles (10)"              returns ("Charles",), 10
       "Alice Charles (10)"        returns ("Alice","Charles"), 10
       "A = B C (12)"              returns ('A','=','B'), 12
    """
    line = line.split()                  # split into words, using blanks to separate
    print(line)
    count = 1                            # default count, if none given
    ballot = [ ]
    for word in line:                    # note that word can't be empty string here
        if word[0] == "(":               # (count) 
            if word[-1]!=")":
                print("Illegal count field %s in line %s"%(word,line))
                sys.exit()
            count = int(word[1:-1])      
        else:                
            ballot += [word]             # candidate
    ballot = tuple(ballot)
    if not ballot_OK(ballot):
        print("Illegal ballot:",line)
        sys.exit()
    return (ballot,count)

def import_ballot(P,ballot,count):
    """
    Add new ballot (or just increment count for ballot) to profile P.
    P = dict mapping ballots to counts
    ballot = tuple of candidates (possibly with "=" signs.
    count = integer multiplicity for ballot
    """
    if not ballot_OK(ballot):
        print("Illegal ballot:", ballot)
        sys.exit()
    if P.__contains__(ballot):
        P[ballot] += count
    else:
        P[ballot] = count

def import_lines(P,lines):
    """
    Update profile P to reflect ballots given in lines.
    Here lines is a sequence of text lines from a file, 
    one ballot per line with (optional) multiplicities given in parentheses,
    and comments following '#' characters.
    """
    for line in lines:
        i = line.find("#")        # note that '#' is a comment char, as in python
        if i>=0:
            line = line[:i]       # removes first "#" and everything after it
        if line != "":
            #print(line)
            (ballot,count) = parse_ballot_line(line)
            import_ballot(P,ballot,count)

def default_params():
    """
    Return dict of default parameters.
    These can be set to different values in an input file by having a line of the
    form:
        ## parametername parametervalue
    e.g.
        ## missing_preferred_less False
    """
    params = { }

    # The parameter  missing_preferred_less
    # if False:
    #    means that ballot  A B  only expressed preference for A over B
    # if True:
    #    means that ballot  A B  also says that A and B preferred to all unlisted candidates.
    params["missing_preferred_less"] = True

    return params

def import_file(filename,P=None,params=None):
    """
    Update profile P corresponding to given file of ballots.
    File has one line per ballot.
    A line may have multipicities, given in parens.
    It is OK to have same ballot more than once; counts just add.
    On any line, # is a comment character; it and later chars are ignored
    A line of the form:
        ## parametername parametervalue
    will set the named parameter to the given value.  Right now,
    the only parameter allowed is "missing_preferred_less", which
    must be True or False.
    """
    if P == None:
        P = { }
    if params == None:
        params = default_params()
    print("Reading ballots from file:",filename)
    file = open(filename,"r")
    text = file.read()
    lines = text.split("\n")
    for line in lines:
        if len(line)>0 and line[0]=="#":               # at least a comment
            print(line)                                 # print it out
            handle_possible_parameter(line,params)     # perhaps setting a parameter
    import_lines(P,lines)
    return P,params

def coerce(x):
    """
    Here input x should be a string (a parameter value).
    Coerce x to nonnegative integer or True/False if possible.
    Else just return x.
    """
    x=x.strip()
    if len(x)==0:
        return x
    if x.lower()=="true":
        return True
    if x.lower()=="false":
        return False
    for c in x:
        if not c.isdigit():
            return x
    return int(x)

def handle_possible_parameter(line,params,printing_wanted=True):
    """
    Input is a line starting with '#'.
    Handle line if it is of the form
        ## parametername parametervalue
    by setting params[parametername] to parametervalue
    If parametervalue is missing, it is set to True
    It is an error if parametername is not already a key in params
    and if new value is not of same type as old value.
    parametervalue coerced to be True/False or an integer if possible
    """
    if len(line)<2:                # too short to be a command
        return
    if line[:2]!="##":             # it's just a comment
        return
    parameter = line[2:].strip()
    if parameter != "":            # something there
        x = parameter.split(" ",1)    # split off parametername from rest (value)
        parametername = x[0]
        if len(x)>1:
            parametervalue = x[1]
        else:
            parametervalue = "True"
        # coerce parameter value if possible to int or True/False
        parametervalue = coerce(parametervalue)
        # check that parameter name is a parameter than can be assigned (i.e. already in params)
        if not params.has_key(parametername):
            print("Error: `%s' is not a parameter than can be set."%parametername)
            sys.exit()
        if type(params[parametername])!=type(parametervalue):
            print("Error: value '%s' does not have proper type for parameter `%s'."%(parametervalue,parametername))
            sys.exit()
        params[parametername] = parametervalue
        print(indent+"Parameter `%s' set to `%s'."%(parametername, parametervalue))

def print_profile(P,election_ID,print_by_decreasing_count=True):
    """
    print a profile of ballots, nicely, sorted in order 
    print in order by decreasing ballot count if print_by_decreasing_count is True, 
    otherwise print in alphabetic order
    """
    if print_by_decreasing_count:
        print("%s: Profile of ballots (with multiplicities), in decreasing order by count:"%election_ID)
        ballots = [(P[ballot],ballot) for ballot in P.keys()]
        ballots = sorted(ballots)
        ballots.reverse()
        ballots = [ b for (cnt,b) in ballots ]
    else:
        print("%s: Profile of ballots (with multiplicities), in sorted (alphabetic) order:"%election_ID)
        ballots = [(ballot,P[ballot]) for ballot in P.keys()]
        ballots = sorted(ballots)
        ballots = [ b for (b,cnt) in ballots ]
    for ballot in ballots:
        line = string.join([ str(alternative) for alternative in ballot ])
        count = P[ballot]
        line += "  (%d)  "%count
        print(indent+line)
    print(indent+"Total count =",number_of_ballots_in_profile(P))

def filter_profile(P,A):
    """
    Return a profile modified by eliminating all candidates not in A.
    Not used now; intended possibly for use later if we say filter profiles
    according to Smith set...
    """
    Pnew = { }                                 
    for ballot in P:
        filtered_ballot = filter( lambda x: x in A, ballot)
        import_ballot(Pnew,filtered_ballot,P[ballot])
    return Pnew

In [4]:
########################################################################################
### PAIRWISE PREFERENCES, MARGINS, AND PAIRWISE COMPARISON GRAPHS
########################################################################################

def pairwise_prefs(A,P,params):
    """
    Return a dict pref that gives for each pair (i,j) of alternatives from A
    the count as to how many voters prefer i to j.
    If params["missing_preferred_less"]==True:
        A short ballot is interpreted as listing the top alternatives only;
        the unlisted alternatives are preferred less than any listed alternative.
    else:
        A short ballot contributes nothing for or against the missing candidates.
    This routine also handles equals signs in ballots.
    """
    pref = { }
    for x in A:
        for y in A:
            pref[(x,y)] = 0
    for ballot in P:
        remaining = A[:]                  # less-preferred candidates remaining
        mentioned = [ ]                   # candidates mentioned so far
        equivalents = [ ]                 # equivalence class, disjoint from mentioned
        last_x = None
        for x in ballot:
            if x != "=":
                remaining.remove(x)
                if last_x == "=":
                    equivalents.append(x)
                else:
                    mentioned.extend(equivalents)
                    equivalents = [ x ]
                for y in mentioned:          # earlier options preferred to x  
                    pref[(y,x)] += P[ballot]
            last_x = x
        if params==None or params["missing_preferred_less"]:  # everything mentioned > everything not
            mentioned.extend(equivalents)
            for x in mentioned:
                for y in remaining:
                    pref[(x,y)] += P[ballot]
    return pref

def pairwise_margins(A,P,params):
    """
    Returns a dict margin that gives for each pair (i,j) of alternatives from A
    the net count of voters that prefer i to j (I.e. the number that prefer i to j
    minus the number that prefer j to i.
    Params is the same as for pairwise_prefs
    """
    pref = pairwise_prefs(A,P,params)
    margin = { }
    for (x,y) in pref:
        margin[(x,y)] = pref[(x,y)] - pref[(y,x)]
    return margin


def print_pairwise_prefs(A,pref,election_ID):
    print("%s: Pairwise preferences (number of voters preferring row over column):"%election_ID)
  

def print_pairwise_margins(A,margin,election_ID):
    print("%s: Pairwise margins (net number of voters preferring row over column):"%election_ID)
    

def save_matrix(filename,A,mat):
    """
    Save matrix mat indexed by pairs of alternatives (from A).
    (This is only used to output margin matrix for possible use within matlab.)
    A = list of candidates/alternatives
    mat = dict mapping pairs of candidates to integers.
    """
    # check that A is sorted, otherwise output is not interpretable.
    assert A == sorted(A)
    file = open(filename,"w")
    # compute width of a cell; big enough for row/column name, or for entry
    width = max( [ len(str(a)) for a in A ] + [ len(str(v)) for v in mat.values() ] )
    for a in A:
        for b in A:
            cnt = 0
            if mat.has_key((a,b)): cnt = mat[(a,b)]
            file.write(string.rjust(str(cnt),width+1)+" ")
        file.write("\n")
    file.close()

In [43]:
import numpy as np

filename = "C:\PhD\Spring22\Game Theory\Project\ex_09.txt"

P,params = import_file(filename)
election_ID = os.path.basename(os.path.splitext(filename)[0])
print(election_ID)
A = alternatives_in_profile(P)
A
margin = pairwise_margins(A,P,params)

keys = margin.keys()

margin_matrix = np.array([margin[i] for i in keys]).reshape(3,3)

margin_matrix

Reading ballots from file: C:\PhD\Spring22\Game Theory\Project\ex_09.txt
# 
# ex_09.txt
# 
['A', 'B', 'C', '(40)']
['B', 'C', 'A', '(30)']
['C', 'A', 'B', '(20)']
['C', 'B', 'A', '(10)']
ex_09


array([[  0,  20, -20],
       [-20,   0,  40],
       [ 20, -40,   0]])

In [44]:
from game_cvxopt import lp_solver

print(lp_solver(margin_matrix))

[0.5000000000217717, 0.24999999992675898, 0.2500000000514694]
